In [1]:
import os
from darkflow.net.build import TFNet
# C:\Users\Gagan Aggawal\Desktop\project\darkflow\darkflow\cfg
options = {"model": "cfg/yolo.cfg", 
           "load": "weights/yolo.weights", 
           "threshold": 0.1}

tfnet = TFNet(options)

/home/gagan/.virtualenvs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gagan/.virtualenvs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gagan/.virtualenvs/ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gagan/.virtualenvs/ml/lib/python3.6/site-packages/tensorflow/

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading weights/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.06553125381469727s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  

In [2]:
def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
            
    return newImage


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import cv2

In [4]:
import pprint as pp

original_img = cv2.imread("./sample_img/sample_multiple_objects.jpg")
original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
results = tfnet.return_predict(original_img)

pp.pprint(results)

[{'bottomright': {'x': 157, 'y': 208},
  'confidence': 0.67079693,
  'label': 'person',
  'topleft': {'x': 61, 'y': 3}},
 {'bottomright': {'x': 286, 'y': 258},
  'confidence': 0.32715744,
  'label': 'person',
  'topleft': {'x': 200, 'y': 38}},
 {'bottomright': {'x': 329, 'y': 187},
  'confidence': 0.2517692,
  'label': 'person',
  'topleft': {'x': 317, 'y': 156}},
 {'bottomright': {'x': 371, 'y': 204},
  'confidence': 0.14781988,
  'label': 'person',
  'topleft': {'x': 354, 'y': 183}},
 {'bottomright': {'x': 524, 'y': 373},
  'confidence': 0.7331757,
  'label': 'person',
  'topleft': {'x': 443, 'y': 139}},
 {'bottomright': {'x': 592, 'y': 378},
  'confidence': 0.22580777,
  'label': 'person',
  'topleft': {'x': 561, 'y': 147}},
 {'bottomright': {'x': 594, 'y': 430},
  'confidence': 0.7661935,
  'label': 'person',
  'topleft': {'x': 501, 'y': 135}},
 {'bottomright': {'x': 596, 'y': 430},
  'confidence': 0.14075229,
  'label': 'person',
  'topleft': {'x': 567, 'y': 363}},
 {'bottomright'

In [5]:
def get_output(frame):
    frame = np.asarray(frame)
    results = tfnet.return_predict(frame)
    new_frame = boxing(frame, results)
    return new_frame

In [ ]:

from moviepy.editor import *
my_clip = VideoFileClip("./sample_video/test_video.mp4")

modifiedClip = my_clip.fl_image( get_output )

modifiedClip.write_videofile("myHolidays_edited.webm")




chunk:   0%|          | 0/34 [00:00<?, ?it/s, now=None]

Moviepy - Building video myHolidays_edited.webm.
MoviePy - Writing audio in myHolidays_editedTEMP_MPY_wvf_snd.ogg


t:   0%|          | 0/38 [00:00<?, ?it/s, now=None]    

MoviePy - Done.
Moviepy - Writing video myHolidays_edited.webm



t:  84%|████████▍ | 32/38 [01:23<00:16,  2.69s/it, now=None]